<a href="https://www.kaggle.com/code/obinna11/notebook7c5a26c9ba?scriptVersionId=99036723" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import re
import lightgbm as lgbm

In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
PassengerId = test["PassengerId"]

y = train["Survived"]
X = train.drop("Survived" , axis = 1)

In [3]:
#feature eng
all_dataset = pd.concat([X, test])
all_dataset["title"] = all_dataset["Name"].apply(lambda row: re.split(", |. |\*", row)[1])
all_dataset["title"] = all_dataset["title"].apply(lambda row: "other" if all_dataset["title"].value_counts()[row]<8 or row == "" else row)

In [4]:
#selecting the needed features
drop_columns = ['Name','PassengerId', 'Cabin','Ticket']
all_dataset.drop(drop_columns, axis =1, inplace= True)

all_dataset['Embarked'].fillna(all_dataset['Embarked'].mode()[0],inplace=True)
categorical_var = ['Sex', 'Embarked', 'title']
ordinal_encoder = OrdinalEncoder()
all_dataset[categorical_var] = ordinal_encoder.fit_transform(all_dataset[categorical_var])

In [5]:
#imputing missing values/ age and fare
knn_imp = KNNImputer(n_neighbors=5)
all_dataset.loc[:, :] = knn_imp.fit_transform(all_dataset)
print(all_dataset.isnull().sum())

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
title       0
dtype: int64


In [6]:
#split dataset back into train and test
X = all_dataset.iloc[:891,:]
test = all_dataset.iloc[891:,:]

In [7]:
#GradientBoosting training and prediction
rf = GradientBoostingClassifier(n_estimators=10)
rf.fit(X,y)
result = rf.predict(test)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('submission.csv', index=False)

#LGBMClassifier training and prediction
rf = lgbm.LGBMClassifier(n_estimators=10)
rf.fit(X,y)
result = rf.predict(test)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('submission2.csv', index=False)

#Only just a marginal imporvement in results between GradientBoosting and LGBMClassifier